In [1]:
import trainer.glue_base as glue_base
import models.sparse_token as sparse
import pickle, importlib
importlib.reload(glue_base)
importlib.reload(sparse)
Glue = glue_base.GlueAttentionApproxTrainer
PICKLE_PATH = "glue_benchmark_wiki.pkl"
TEX_PATH = "glue_benchmark_wiki.tex"

In [2]:
subsets = ["cola","mnli","mrpc","qnli","qqp","rte","sst2","stsb","wnli",]
kss = [
    0.1, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.999, 'dynamic', 
    'dynamic:avg:avg:true', 'dynamic:avg:avg:false', 'dynamic:avg:max:true', 'dynamic:avg:max:false',
    'dynamic:max:avg:true', 'dynamic:max:avg:false', 'dynamic:max:max:true', 'dynamic:max:max:false',
]
kss = ['dynamic']
sparse.benchmark_reset()
# subsets = ["mrpc"]
# kss = ['dynamic:avg:avg:f',0.1]

def get_score(score):
    if 'accuracy' in score:
        return score['accuracy'], "acc"
    first_metric = list(score.keys())[0]
    return score[first_metric], first_metric

results = {}
i = 0
for subset in subsets:
    trainer = Glue(dataset=subset, factor=16, batch_size=-1, device=0, wiki_train=True)
    trainer.load()
    scores = {}
    metric_name = ""
    bert_score, metric_name = get_score(trainer.eval_base_model())
    scores['bert'] = f'{bert_score:.5f}'
    for ks in kss:
        sparse.benchmark_reset()
        sparse_score, _ = get_score(trainer.eval_sparse_model(ks=ks))
        if isinstance(ks, str) and ks.startswith('dynamic'):
            est_k = sparse.benchmark_get_average('est_k')
            scores[str(ks)] = f'{sparse_score:.5f} (k:{est_k:.2f})'
        else:
            scores[str(ks)] = f'{sparse_score:.5f}'
        i += 1
        count = len(subsets) * len(kss)
        print(f'{i}/{count}')
    results[f"{subset} ({metric_name})"] = scores

with open(PICKLE_PATH, 'wb') as f:
    pickle.dump(results, f)

sparse.benchmark_report()

Trainer: cola


Reusing dataset glue (/home/ainl/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at /home/ainl/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-f0e04bff3297c11c.arrow
Loading cached processed dataset at /home/ainl/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-b9e7d91153cc59cf.arrow
Reusing dataset glue (/home/ainl/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at /home/ainl/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-7e570c95fb4f22e4.arrow
Loading cached processed dataset at /home/ainl/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cac

Trainer: Save checkpoint path saves/glue-cola-16-wiki.pth


eval: 100%|██████████| 131/131 [00:01<00:00, 72.15it/s]


metric score {'matthews_correlation': 0.5338774230813111}
avg occupy 0.34522852946633237


...

Trainer: Save checkpoint path saves/glue-wnli-16-wiki.pth


eval: 100%|██████████| 9/9 [00:00<00:00, 58.27it/s]


metric score {'accuracy': 0.5633802816901409}
avg occupy 0.3578609244533317


eval: 100%|██████████| 9/9 [00:00<00:00, 36.04it/s]

metric score {'accuracy': 0.5633802816901409}
avg occupy 0.3578609244533317
9/9
est_k: (108, 58.038095238095245), avg: 0.5373897707231041


In [14]:
import pickle
import pandas as pd

with open(PICKLE_PATH, 'rb') as f:
    results = pickle.load(f)
with open('glue_benchmark.pkl', 'rb') as f:
    results_original = pickle.load(f)

def convert_data_to_results(results):
    data = []
    subsets = list(results.keys())
    factors = list(results[subsets[0]].keys())
    for factor in factors:             
        row = []
        for subset in subsets:
            row.append(results[subset][factor])
        data.append(row)
    return data, factors

data, factors = convert_data_to_results(results)
data_origin, _ = convert_data_to_results(results_original)

df = pd.DataFrame(data + [data_origin[9]], columns=subsets, index=factors+["dynamic:original"])
tex = df.to_latex()
with open(TEX_PATH, 'w') as f:
    f.write(tex)
df

/tmp/ipykernel_89525/225414492.py:24: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  tex = df.to_latex()


,cola (matthews_correlation),mnli (acc),mrpc (acc),qnli (acc),qqp (acc),rte (acc),sst2 (acc),stsb (pearson),wnli (acc)
bert,0.53388,0.84198,0.84406,0.91543,0.90908,0.72563,0.92431,0.88047,0.56338
dynamic,0.53388 (k:0.48),0.81559 (k:0.30),0.74667 (k:0.66),0.90445 (k:0.41),0.90893 (k:0.43),0.70758 (k:0.38),0.92202 (k:0.69),0.86572 (k:0.49),0.56338 (k:0.54)
dynamic:original,0.53388 (k:0.48),0.84004 (k:0.33),0.83942 (k:0.71),0.91287 (k:0.44),0.90920 (k:0.45),0.72563 (k:0.55),0.92317 (k:0.73),0.88043 (k:0.51),0.56338 (k:0.63)


# pre data augment

cola (matthews_correlation)	mnli (acc)	mrpc (acc)	qnli (acc)	qqp (acc)	rte (acc)	sst2 (acc)	stsb (pearson)	wnli (acc)
bert	0.53388	0.84198	0.84406	0.91543	0.90908	0.72563	0.92431	0.88047	0.56338
dynamic	0.53388 (k:0.48)	0.81559 (k:0.30)	0.74667 (k:0.66)	0.90445 (k:0.41)	0.90893 (k:0.43)	0.70758 (k:0.38)	0.92202 (k:0.69)	0.86572 (k:0.49)	0.56338 (k:0.54)
dynamic:original	0.53388 (k:0.48)	0.84004 (k:0.33)	0.83942 (k:0.71)	0.91287 (k:0.44)	0.90920 (k:0.45)	0.72563 (k:0.55)	0.92317 (k:0.73)	0.88043 (k:0.51)	0.56338 (k:0.63)